In [ ]:
%load_ext autoreload
%autoreload

import torch

from pathlib import Path
import sys

# allows importing from the src folder
root_path = Path().absolute().parents[0]
if root_path not in sys.path:
    sys.path.append(root_path.as_posix())

import src.utils.math as math_utils

# Test finding corners, calculating area

In [ ]:
bb = torch.tensor([0,0,4,3])

corners = math_utils.bounding_box_corners(bb)
assert torch.allclose(
    corners,
    torch.tensor([
        [0,0],
        [4,0],
        [4,3],
        [0,3]
    ])
)
print(corners[0,:,:])

area = math_utils.bounding_box_area(bb).item()

assert area == 12

bbs = torch.concat([bb,bb])
areas = math_utils.bounding_box_area(bbs)

assert len(areas.shape) == 1
assert torch.allclose(
    areas - torch.tensor([12,12], dtype=float),
    torch.zeros(2, dtype=float)
)

In [ ]:


bbs = torch.rand([5,4])

corners = math_utils.bounding_box_corners(bbs)
print(corners.shape)

print(f"{bbs=}")
print(corners)


# Test corner inclusion function

In [ ]:

bbs1 = torch.tensor([
    [0,0,2,2],
    [1,1,3,3]
], dtype=torch.float32)

result = math_utils.included_corners(bbs1, bbs1)
# box's corners are within itself, and for the above,
# first box has it bottom right corner inside the second,
# second has top left corner inside first
assert [result[i].sum().item() for i in range(4)] == [3,2,3,2]
assert result[2,0,1]

# top left corner inside both of bbs1's
bbs2 = torch.tensor([
    [1.5,1.5,4,4]
])

print(f"{bbs1.shape=}")
print(f"{bbs2.shape=}")

result = math_utils.included_corners(bbs1, bbs2)

print(result)
print(result.shape)
assert torch.allclose(
    torch.tensor(result.shape),
    torch.tensor([4,1,2])
)
assert [result[i].sum().item() for i in range(4)] == [2,0,0,0]

# NOTE: reverse of the above (switch bbs1 and bbs2)
result = math_utils.included_corners(bbs2, bbs1)
print(result)
print(result.shape)
assert torch.allclose(
    torch.tensor(result.shape),
    torch.tensor([4,2,1])
)



# top left inside first, bottom right inside second
bbs2 = torch.concat(
    [
        bbs2.flatten(),
        torch.tensor([0.25,0.25,2.5,2.5])
    ]
).reshape([2,4])

result = math_utils.included_corners(bbs1, bbs2)
print(result)

assert [result[i].sum().item() for i in range(4)] == [2+1,0,1,0]
assert result[2,1,1]


tensr = torch.tensor([0,0,1,1]).reshape([1,4])
assert torch.all(math_utils.included_corners(tensr, tensr))

# Test intersection area functions

In [ ]:
bbs1 = torch.tensor([
    [10,10,12,12]
], dtype=torch.float32).reshape([-1,4])

bbs2 = torch.tensor([
    [10,10,12,12], # exact overlap, area = 4
    [11,11,13,13], # top left corner in, area = 1
    [11.5,9.5,12.5,10.5], # bottom left corner in, area = 0.25
    [9,10.5,11.5,11.5], # right side in, area = 1.5
    [11,11,11.5,11.5], # completely inside, area = 0.25
    [9,9,13,13], # completely contains, area = 4
    [1,1,2,2], # no overlap, area = 0
])


one_to_two = math_utils.intersection(bbs1, bbs2)
assert torch.allclose(one_to_two.flatten(), torch.tensor([4,1,0.25,1.5,0.25,0,0]))

two_to_one = math_utils.intersection(bbs2, bbs1)
assert torch.allclose(two_to_one.flatten(), torch.tensor([4,1,0.25,0,0,4,0]))

symmetric = math_utils.symmetric_intersection(bbs1, bbs2)
assert torch.allclose(symmetric.flatten(), torch.tensor([4,1,0.25,1.5,0.25,4,0]))

# Test intersection-over-union

In [ ]:


bbs1 = torch.tensor([
    [10,10,12,12],
    [11,11,12,12]
], dtype=torch.float32)

bbs2 = torch.tensor([
    [10,10,12,12],
    [11,11,13.5,13.5],
    [0,0,1,1]
])

expected = torch.tensor([
    [4/(4+4-4), 1/(4+1-1)],
    [1/(4+2.5**2-1), 1/(1+2.5**2-1)],
    [0,0]
])

actual = math_utils.intersection_over_union(bbs1,bbs2)
assert torch.allclose(expected, actual)

assert torch.allclose(actual, math_utils.intersection_over_union(bbs2,bbs1).T)